In [1]:
# NormStats

In [2]:
import sys; sys.path.append('../..')
from abslithist.words import *

In [3]:
dforig = get_orignorms()
dforig

,Abs-Conc.LSN-Hapt,Abs-Conc.LSN-Imag,Abs-Conc.MRC-Conc,Abs-Conc.MRC-Imag,Abs-Conc.MT-Conc,Abs-Conc.PAV-Conc,Abs-Conc.PAV-Imag,Abs-Conc.Median
word,,,,,,,,
I,NaN,NaN,NaN,NaN,0.950954,NaN,NaN,0.950954
Inf,NaN,NaN,NaN,NaN,-1.584783,NaN,NaN,-1.584783
a.lgebra,NaN,NaN,NaN,NaN,NaN,-0.050006,0.151973,0.050983
aardvark,0.630471,1.413671,NaN,NaN,1.690955,NaN,NaN,1.413671
aback,-0.822187,-0.043892,NaN,NaN,-1.298649,NaN,NaN,-0.822187
...,...,...,...,...,...,...,...,...
zsuds,NaN,NaN,0.947265,1.058508,NaN,NaN,NaN,1.002887
zteeth,NaN,NaN,1.473838,1.478364,NaN,NaN,NaN,1.476101
ztweezers,NaN,NaN,1.243463,1.551383,NaN,NaN,NaN,1.397423


In [4]:
dforig.corr().median().median()


0.8556618987515798

In [5]:
col1=dforig.columns[0]
col2=dforig.columns[1]
x=dforig[col1]
y=dforig[col2]
shared = sorted([x for x in (set(x.dropna().index) & set(y.dropna().index)) if type(x)==str and x==x.lower()])
x.loc[shared]

word
aardvark     0.630471
aback       -0.822187
abacus       2.919588
abandon     -0.822187
abandoned   -1.082577
               ...   
zoologist   -0.256373
zoology     -0.997683
zoom        -0.706616
zoophobia   -0.207645
zucchini     1.888727
Name: Abs-Conc.LSN-Hapt, Length: 35745, dtype: float64

In [6]:
# Lin reg
from scipy.stats import linregress
import random
from tqdm import tqdm

def cord(obj):
    x,y,xname,yname,nrun,batchsize=obj
    shared = [x for x in (set(x.dropna().index) & set(y.dropna().index)) if type(x)==str and x==x.lower()]
    
    ld=[]
    for n in range(nrun):
        sharedrun=random.sample(shared,batchsize)
        xrun = x.loc[sharedrun]
        yrun = y.loc[sharedrun]
        d={'x':xname, 'y':yname}
        d['slope'], d['intercept'], d['r_value'], d['p_value'], d['std_err'] = linregress(xrun, yrun)
        d['r^2']=d['r_value']**2
        d['n']=len(sharedrun)
        ld.append(d)
#     d=dict(pd.DataFrame(ld).groupby(['x','y']).median().reset_index())
    
    return ld

In [7]:
# objs = [
#     (dforig[col1],dforig[col2],col1,col2)
#     for col1 in dforig.columns
#     for col2 in dforig.columns
#     if col1<col2
# ]
# len(objs)

In [8]:
# import lltk
# res = lltk.pmap(cord, objs,num_proc=1)
# df = pd.DataFrame(res)
# df.sort_values('r^2')

In [9]:
# dict(corall.median())

## Into a function

In [10]:
def cornorms(dforig,nrun=100,batchsize=500):
    objs = [
        (dforig[col1],dforig[col2],col1,col2,nrun,batchsize)
        for col1 in dforig.columns
        for col2 in dforig.columns
        if col1!=col2
    ]
    
    import lltk
    res = lltk.pmap(cord, objs,num_proc=4)
    df = pd.DataFrame([d for ld in res for d in ld])
#     df = pd.DataFrame(res)#[d for ld in res for d in ld])
    print('Median r^2 =',df['r^2'].median())
#     return df.sort_values('r^2')
    return df

In [16]:
cororig = cornorms(dforig)
cororig

Mapping cord() [x4]: 100%|██████████| 56/56 [00:02<00:00, 22.58it/s]


Median r^2 = 0.6654793151264684


,x,y,slope,intercept,r_value,p_value,std_err,r^2,n
0,Abs-Conc.LSN-Hapt,Abs-Conc.LSN-Imag,0.475230,0.070424,0.474503,1.944057e-29,0.039506,0.225153,500
1,Abs-Conc.LSN-Hapt,Abs-Conc.LSN-Imag,0.512304,0.028745,0.502580,2.322987e-33,0.039490,0.252586,500
2,Abs-Conc.LSN-Hapt,Abs-Conc.LSN-Imag,0.414324,-0.031549,0.426084,1.799559e-23,0.039421,0.181547,500
3,Abs-Conc.LSN-Hapt,Abs-Conc.LSN-Imag,0.361024,0.024159,0.376835,2.568132e-18,0.039766,0.142005,500
4,Abs-Conc.LSN-Hapt,Abs-Conc.LSN-Imag,0.494702,0.049710,0.477894,6.822424e-30,0.040747,0.228383,500
...,...,...,...,...,...,...,...,...,...
5595,Abs-Conc.Median,Abs-Conc.PAV-Imag,1.000328,-0.235336,0.926082,6.147250e-213,0.018264,0.857628,500
5596,Abs-Conc.Median,Abs-Conc.PAV-Imag,1.031943,-0.261178,0.934481,1.632126e-225,0.017617,0.873254,500
5597,Abs-Conc.Median,Abs-Conc.PAV-Imag,1.016487,-0.238806,0.933979,1.021990e-224,0.017427,0.872317,500
5598,Abs-Conc.Median,Abs-Conc.PAV-Imag,1.010085,-0.228153,0.941530,1.966163e-237,0.016198,0.886478,500


In [18]:
cororig.groupby('x').mean().sort_values('r^2')

,slope,intercept,r_value,p_value,std_err,r^2,n
x,,,,,,,
Abs-Conc.LSN-Hapt,0.474138,-0.039638,0.555927,2.494697e-20,0.031746,0.316326,500
Abs-Conc.LSN-Imag,0.546498,-0.096558,0.580212,5.252436e-17,0.034394,0.345683,500
Abs-Conc.MRC-Imag,0.839691,0.039765,0.772152,1.950493e-24,0.029152,0.623855,500
Abs-Conc.MT-Conc,0.703908,-0.239427,0.780186,1.265717e-25,0.023915,0.632964,500
Abs-Conc.MRC-Conc,0.823679,0.176949,0.799684,4.156243e-31,0.025191,0.667784,500
Abs-Conc.PAV-Imag,0.829798,0.308762,0.805708,8.663094e-37,0.025590,0.671546,500
Abs-Conc.PAV-Conc,0.823254,0.314399,0.809281,1.431838e-39,0.024070,0.680337,500
Abs-Conc.Median,0.966768,-0.092145,0.861422,2.452235e-63,0.024855,0.748334,500


In [12]:
round(cororig.median(),2)

slope          0.74
intercept     -0.06
r_value        0.81
p_value        0.00
std_err        0.03
r^2            0.66
n            500.00
dtype: float64

In [13]:
cororig.describe()

,slope,intercept,r_value,p_value,std_err,r^2,n
count,2800.000000,2800.000000,2800.000000,2.800000e+03,2800.000000,2800.000000,2800.0
mean,0.718271,-0.076456,0.744792,1.939371e-19,0.025429,0.584900,500.0
std,0.211837,0.228221,0.173769,7.509389e-18,0.008815,0.252479,0.0
min,0.327761,-0.574883,0.353421,0.000000e+00,0.005867,0.124907,500.0
25%,0.518519,-0.233177,0.562627,1.467691e-189,0.018665,0.316549,500.0
50%,0.740650,-0.055220,0.813718,1.988970e-119,0.025467,0.662137,500.0
75%,0.871956,0.021228,0.907362,4.397285e-43,0.032296,0.823306,500.0
max,1.057875,0.602400,0.991192,3.719379e-16,0.045818,0.982462,500.0


In [19]:
# stop

In [20]:
corvec=cornorms(get_vecnorms())
corvec

Mapping cord() [x4]: 100%|██████████| 2256/2256 [12:50<00:00,  2.93it/s]


Median r^2 = 0.7730828130381499


,x,y,slope,intercept,r_value,p_value,std_err,r^2,n
0,Abs-Conc.LSN-Hapt.C16,Abs-Conc.LSN-Imag.C16,0.921873,-0.004241,0.907620,7.351616e-190,0.019107,0.823774,500
1,Abs-Conc.LSN-Hapt.C16,Abs-Conc.LSN-Imag.C16,0.895841,-0.024719,0.921461,1.225614e-206,0.016924,0.849091,500
2,Abs-Conc.LSN-Hapt.C16,Abs-Conc.LSN-Imag.C16,0.919498,-0.030958,0.927938,1.386896e-215,0.016551,0.861070,500
3,Abs-Conc.LSN-Hapt.C16,Abs-Conc.LSN-Imag.C16,0.927854,-0.033081,0.915944,1.325967e-199,0.018217,0.838953,500
4,Abs-Conc.LSN-Hapt.C16,Abs-Conc.LSN-Imag.C16,0.916405,-0.039102,0.930168,7.371442e-219,0.016208,0.865212,500
...,...,...,...,...,...,...,...,...,...
225595,Abs-Conc.PAV-Imag.median,Abs-Conc.PAV-Conc.median,0.965489,0.051741,0.974235,0.000000e+00,0.010016,0.949134,500
225596,Abs-Conc.PAV-Imag.median,Abs-Conc.PAV-Conc.median,0.956716,0.036235,0.969286,1.569531e-305,0.010878,0.939516,500
225597,Abs-Conc.PAV-Imag.median,Abs-Conc.PAV-Conc.median,0.983967,0.046498,0.970282,4.830342e-309,0.010996,0.941448,500
225598,Abs-Conc.PAV-Imag.median,Abs-Conc.PAV-Conc.median,0.969827,0.045024,0.972089,1.000114e-315,0.010489,0.944957,500


In [24]:
round(corvec.median(),2)

slope          0.89
intercept      0.00
r_value        0.88
p_value        0.00
std_err        0.02
r^2            0.77
n            500.00
dtype: float64

In [29]:
corvec#.groupby('x').mean().sort_values('r^2')

,x,y,slope,intercept,r_value,p_value,std_err,r^2,n
0,Abs-Conc.LSN-Hapt.C16,Abs-Conc.LSN-Imag.C16,0.921873,-0.004241,0.907620,7.351616e-190,0.019107,0.823774,500
1,Abs-Conc.LSN-Hapt.C16,Abs-Conc.LSN-Imag.C16,0.895841,-0.024719,0.921461,1.225614e-206,0.016924,0.849091,500
2,Abs-Conc.LSN-Hapt.C16,Abs-Conc.LSN-Imag.C16,0.919498,-0.030958,0.927938,1.386896e-215,0.016551,0.861070,500
3,Abs-Conc.LSN-Hapt.C16,Abs-Conc.LSN-Imag.C16,0.927854,-0.033081,0.915944,1.325967e-199,0.018217,0.838953,500
4,Abs-Conc.LSN-Hapt.C16,Abs-Conc.LSN-Imag.C16,0.916405,-0.039102,0.930168,7.371442e-219,0.016208,0.865212,500
...,...,...,...,...,...,...,...,...,...
225595,Abs-Conc.PAV-Imag.median,Abs-Conc.PAV-Conc.median,0.965489,0.051741,0.974235,0.000000e+00,0.010016,0.949134,500
225596,Abs-Conc.PAV-Imag.median,Abs-Conc.PAV-Conc.median,0.956716,0.036235,0.969286,1.569531e-305,0.010878,0.939516,500
225597,Abs-Conc.PAV-Imag.median,Abs-Conc.PAV-Conc.median,0.983967,0.046498,0.970282,4.830342e-309,0.010996,0.941448,500
225598,Abs-Conc.PAV-Imag.median,Abs-Conc.PAV-Conc.median,0.969827,0.045024,0.972089,1.000114e-315,0.010489,0.944957,500


In [32]:
corvec['x_period']=corvec.x.apply(lambda x: x.split('.')[-1])
corvec['x_source']=corvec.x.apply(lambda x: x.split('.')[-2])
corvec.groupby('x_period').mean().sort_values('r^2')

,slope,intercept,r_value,p_value,std_err,r^2,n
x_period,,,,,,,
C16,0.775914,-0.062772,0.827638,1.665950e-57,0.022184,0.692261,500
C20,0.892441,0.092994,0.848417,7.891270e-55,0.023829,0.727580,500
C17,0.832388,-0.081698,0.860634,9.637579e-67,0.021009,0.744967,500
C18,0.844735,0.012879,0.863856,1.966538e-60,0.021098,0.751095,500
C19,0.915107,0.050047,0.878516,3.451237e-67,0.021340,0.776782,500
median,0.957492,0.003188,0.927879,1.436494e-110,0.016813,0.861926,500


In [33]:
corvec.groupby('x_source').mean().sort_values('r^2')

,slope,intercept,r_value,p_value,std_err,r^2,n
x_source,,,,,,,
LSN-Hapt,0.848810,-0.041024,0.847391,1.676010e-57,0.022943,0.723561,500
LSN-Imag,0.869307,0.007844,0.863917,4.761760e-57,0.021588,0.752245,500
PAV-Imag,0.869522,0.015392,0.865079,5.396475e-59,0.021366,0.754382,500
MRC-Imag,0.865504,0.002324,0.865207,1.428493e-57,0.021266,0.754462,500
MT-Conc,0.871252,0.018602,0.866821,1.012255e-54,0.021206,0.757489,500
PAV-Conc,0.875558,0.008361,0.875776,3.460844e-62,0.020225,0.772917,500
Median,0.879220,-0.000407,0.879150,3.215233e-65,0.019926,0.778797,500
MRC-Conc,0.878263,0.008424,0.879246,3.421842e-56,0.019845,0.778963,500


In [34]:
round(corvec.describe(),4)

,slope,intercept,r_value,p_value,std_err,r^2,n
count,225600.0000,225600.0000,225600.0000,225600.0,225600.0000,225600.0000,225600.0
mean,0.8697,0.0024,0.8678,0.0,0.0210,0.7591,500.0
std,0.0938,0.0998,0.0774,0.0,0.0069,0.1321,0.0
min,0.5380,-0.3582,0.6003,0.0,0.0024,0.3603,500.0
25%,0.8168,-0.0483,0.8063,0.0,0.0161,0.6502,500.0
50%,0.8869,0.0006,0.8793,0.0,0.0213,0.7731,500.0
75%,0.9418,0.0494,0.9331,0.0,0.0259,0.8707,500.0
max,1.0625,0.4387,0.9985,0.0,0.0424,0.9971,500.0


In [ ]:
corall=cornorms(get_allnorms())
corall

Mapping cord() [x4]:  29%|██▉       | 889/3080 [05:37<13:26,  2.72it/s]

In [ ]:
round(corall.median(),2)

In [ ]:
# corall.min()

In [ ]:
orig_v_vec = corall[
    (corall.x.str.endswith('orig') & ~corall.y.str.endswith('orig')) | (~corall.x.str.endswith('orig') & corall.y.str.endswith('orig'))
]
# orig_v_vec

In [ ]:
round(orig_v_vec.median(),2)

In [ ]:
orig_v_vec_no_hapt=orig_v_vec[~orig_v_vec.x.str.contains('Hapt') & ~orig_v_vec.y.str.contains('Hapt')]
orig_v_vec_no_lsn=orig_v_vec[~orig_v_vec.x.str.contains('LSN') & ~orig_v_vec.y.str.contains('LSN')]
# orig_v_vec_no_hapt

In [ ]:
round(orig_v_vec_no_hapt.median(),2)

In [ ]:
round(orig_v_vec_no_lsn.median(),2)